# Optimization

## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import importlib
import numpy as np
import pandas as pd

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

from src import config
import src.optimization_utils as ou
import src.solutions as s
import src.display as disp
from src.problem import SpineProblem

# Reload to pick up any changes
importlib.reload(ou)
importlib.reload(s)
importlib.reload(disp)

pd.set_option('display.max_columns', None)

## Load Models

In [2]:
# Mechanical failure model
mech_fail_bundle = ou.load_model_bundle(config.MECH_FAIL_MODEL)

# Delta models from notebook 03
L4S1_bundle = ou.load_model_bundle(config.L4S1_MODEL)
LL_bundle = ou.load_model_bundle(config.LL_MODEL)
T4PA_bundle = ou.load_model_bundle(config.T4PA_MODEL)
L1PA_bundle = ou.load_model_bundle(config.L1PA_MODEL)

# Delta models from notebook 04
SVA_bundle = ou.load_model_bundle(config.SVA_MODEL)
SS_bundle = ou.load_model_bundle(config.SS_MODEL)
GT_bundle = ou.load_model_bundle(config.GLOBAL_TILT_MODEL)

print("Loaded models:")
print(f"  - Mechanical failure: {mech_fail_bundle.get('model_name', 'N/A')}")
print(f"  - L4S1: {L4S1_bundle.get('model_name', 'N/A')}")
print(f"  - LL: {LL_bundle.get('model_name', 'N/A')}")
print(f"  - T4PA: {T4PA_bundle.get('model_name', 'N/A')}")
print(f"  - L1PA: {L1PA_bundle.get('model_name', 'N/A')}")
print(f"  - SVA: {SVA_bundle.get('model_name', 'N/A')}")
print(f"  - SS: {SS_bundle.get('model_name', 'N/A')}")
print(f"  - Global Tilt: {GT_bundle.get('model_name', 'N/A')}")

Loaded models:
  - Mechanical failure: mech_fail_logreg
  - L4S1: L4S1_ridge_reg
  - LL: LL_ridge_reg
  - T4PA: T4PA_ridge_reg
  - L1PA: L1PA_ridge_reg
  - SVA: XGBRegressor_delta_SVA
  - SS: XGBRegressor_delta_SS
  - Global Tilt: XGBRegressor_delta_GlobalTilt


## Optimization Configuration

In [3]:
# =============================================================================
# COMPOSITE SCORE WEIGHTS - Adjust these to change optimization priorities
# =============================================================================
# All weights should sum to 1.0 for proper scaling
# Lower composite score = better outcome

WEIGHTS = {
    "w1": 1/6,  # GAP Score (normalized 0-100)
    "w2": 1/6,  # L1PA penalty (mismatch from ideal)
    "w3": 1/6,  # L4S1 penalty (ideal range 35-45)
    "w4": 1/6,  # T4L1PA penalty (T4PA - L1PA mismatch)
    "w5": 1/6,  # LL penalty (mismatch from ideal LL)
    "w6": 1/6,  # GAP category improvement penalty
}

WEIGHT_LABELS = {
    "w1": "GAP Score",
    "w2": "L1PA penalty",
    "w3": "L4S1 penalty",
    "w4": "T4L1PA penalty",
    "w5": "LL penalty",
    "w6": "GAP category improvement",
}

print("Composite Score Weights:")
for k, v in WEIGHTS.items():
    print(f"  {k}: {v:.4f}  ({WEIGHT_LABELS[k]})")
print(f"  Total: {sum(WEIGHTS.values()):.4f}")

Composite Score Weights:
  w1: 0.1667  (GAP Score)
  w2: 0.1667  (L1PA penalty)
  w3: 0.1667  (L4S1 penalty)
  w4: 0.1667  (T4L1PA penalty)
  w5: 0.1667  (LL penalty)
  w6: 0.1667  (GAP category improvement)
  Total: 1.0000


In [4]:
delta_bundles = {
    "L4S1": L4S1_bundle,
    "LL": LL_bundle,
    "T4PA": T4PA_bundle,
    "L1PA": L1PA_bundle,
    "SS": SS_bundle,
    "GlobalTilt": GT_bundle,
    "SVA": SVA_bundle,
}

print("Delta model bundles loaded:", list(delta_bundles.keys()))

Delta model bundles loaded: ['L4S1', 'LL', 'T4PA', 'L1PA', 'SS', 'GlobalTilt', 'SVA']


In [5]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

In [6]:
print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

# Use column names from config
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=config.DECISION_VAR_NAMES))

UIV_CHOICES: ['Hook', 'PS', 'FS']
xl: [0 0 0 0 0 0 1 2 0]
xu: [2 1 5 1 1 1 6 4 1]
    uiv_code  num_levels_cat_code  num_interbody_fusion_levels  ALIF  XLIF  \
xl         0                    0                            0     0     0   
xu         2                    1                            5     1     1   

    TLIF  num_rods  num_pelvic_screws  osteotomy  
xl     0         1                  2          0  
xu     1         6                  4          1  


## Test Patient w fixed parameters

In [7]:
# Load patient from dataset by ID
# PATIENT_ID = 1206016
PATIENT_ID = 1176294
patient_fixed = ou.load_patient_data(patient_id=PATIENT_ID)

# Alternatively can load patient by index
# patient_fixed = ou.load_patient_data(index=0)

print(f"Loaded patient with ID {PATIENT_ID}")
print(f"Total patients in dataset: {ou.get_patient_count()}")
print("\nPatient fixed parameters:")
for k, v in patient_fixed.items():
    print(f"  {k}: {v}")

Loaded patient with ID 1176294
Total patients in dataset: 277

Patient fixed parameters:
  age: 71
  sex: MALE
  bmi: 28.02
  C7CSVL_preop: 26.2
  SVA_preop: 83.6
  T4PA_preop: 27.6
  L1PA_preop: 18.7
  LL_preop: 22.7
  L4S1_preop: 15.8
  PT_preop: 33.1
  PI_preop: 50.4
  SS_preop: 17.3
  cobb_main_curve_preop: 21.5
  FC_preop: 17.4
  tscore_femneck_preop: None
  HU_UIV_preop: 78.0
  HU_UIVplus1_preop: 87.0
  HU_UIVplus2_preop: 98.0
  gap_category: SD
  gap_score_preop: 10.0
  GlobalTilt_preop: 40.3


## Build optimization problem

**Objective:** Minimize composite score (lower = better patient outcomes)

**Constraints:** 
- If `num_interbody_fusion_levels > 0`, at least one fusion type (`ALIF`, `XLIF`, or `TLIF`) must be selected.

In [8]:
problem = SpineProblem(
    patient_fixed=patient_fixed,
    delta_bundles=delta_bundles,
    xl=xl,
    xu=xu,
    weights=WEIGHTS
)

## Run GA and view results

In [9]:
algorithm = GA(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    get_termination("n_gen", 20),
    seed=42,
    verbose=True,
    save_history=True
)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       99 |  0.000000E+00 |  0.1919191919 |  8.6148157597 |  1.4078026312
     2 |      199 |  0.000000E+00 |  0.000000E+00 |  5.1507475831 |  1.4078026312
     3 |      299 |  0.000000E+00 |  0.000000E+00 |  1.8561876227 |  1.4078026312
     4 |      399 |  0.000000E+00 |  0.000000E+00 |  1.6438337067 |  1.4078026312
     5 |      499 |  0.000000E+00 |  0.000000E+00 |  1.5304979698 |  1.3767859473
     6 |      599 |  0.000000E+00 |  0.000000E+00 |  1.4639254618 |  1.3767859473
     7 |      699 |  0.000000E+00 |  0.000000E+00 |  1.4261118442 |  1.3767859473
     8 |      799 |  0.000000E+00 |  0.000000E+00 |  1.4158729643 |  1.3767859473
     9 |      899 |  0.000000E+00 |  0.000000E+00 |  1.4055171776 |  1.3767859473
    10 |      999 |  0.000000E+00 |  0.000000E+00 |  1.3926784145 |  1.3767859473
    11 |     1099 |  0.000000E+00 |  0.000000E+00 |  1.3819551180 |  1.3767859473
    12 |     119

## Actual surgical plan & outcome for comparison

In [10]:
df_actual = disp.display_actual_outcomes(PATIENT_ID, patient_fixed)
display(df_actual)

ACTUAL SURGICAL PLAN (WHAT WAS PERFORMED)
  UIV_implant: FS
  num_levels_cat: lower
  num_interbody_fusion_levels: 4.0
  ALIF: 1
  XLIF: 1
  TLIF: 0
  num_rods: 4.0
  num_pelvic_screws: 2.0
  osteotomy: 1.0

ALIGNMENT PARAMETERS: PREOP → POSTOP (ACTUAL)


,Parameter,Preop,Delta (actual),Postop (actual)
0,LL,22.7,43.2,65.9
1,SS,17.3,18.3,35.6
2,L4S1,15.8,18.1,33.9
3,GlobalTilt,40.3,-27.7,12.6
4,T4PA,27.6,-23.3,4.3
5,L1PA,18.7,-12.8,5.9
6,PI,50.4,-0.7,49.7
7,PT,33.1,-19.0,14.1
8,SVA,83.6,-84.6,-1.0
9,PI-LL,27.7 ⚠,-43.9,-16.2 ⚠


## Show best solution

In [11]:
best_x = np.asarray(res.X).astype(int)
result = ou.evaluate_solution(
    best_x, 
    patient_fixed, 
    delta_bundles, 
    mech_fail_bundle,
    weights=WEIGHTS
)

df_comparison = disp.display_optimized_solution(result, patient_fixed)
display(df_comparison)

BEST SOLUTION SUMMARY (OPTIMIZED)

Composite Score: 1.3768 (lower is better)
Mechanical Failure Probability: 27.2%

Surgical Plan:
  UIV_implant: Hook
  num_levels_cat: lower
  num_interbody_fusion_levels: 4
  ALIF: 1
  XLIF: 0
  TLIF: 0
  num_rods: 6
  num_pelvic_screws: 4
  osteotomy: 1

ALIGNMENT PARAMETERS: PREOP → POSTOP (PREDICTED)


,Parameter,Preop,Delta (pred),Postop (pred)
0,LL,22.7,33.4,56.1
1,SS,17.3,15.2,32.5
2,L4S1,15.8,26.4,42.2
3,GlobalTilt,40.3,-27.1,13.2
4,T4PA,27.6,-17.8,9.8
5,L1PA,18.7,-12.1,6.6
6,SVA,83.6,-74.1,9.5
7,PI,50.4,-,50.4
8,PT,33.1,-15.2,17.9
9,PI-LL,27.7 ⚠,-33.4,-5.7 ⚠


## Getting multiple solutions

Extracts diverse surgical plans from the GA optimization history:
1. Collects top candidates across all generations
2. Filters to solutions within `score_tolerance` of best score
3. Buckets by `(UIV_implant, ALIF, XLIF, TLIF)` to ensure variety in implant/fusion types
4. Returns 'top_n` unique plans with full evaluation (postop values, GAP score, mech fail prob)

In [12]:
top12_solutions = s.get_diverse_solutions(
    res=res,
    top_n=12,
    top_per_gen=50,
    score_tolerance=2,  # Include solutions within 2 points of best
    bucket_cols=("UIV_implant", "ALIF", "XLIF", "TLIF"),
    n_per_bucket=1,
    patient_fixed=patient_fixed,
    delta_bundles=delta_bundles,
    mech_fail_bundle=mech_fail_bundle,
    weights=WEIGHTS,
)

_ = disp.display_multiple_solutions(top12_solutions, patient_fixed)
# top12_solutions

SOLUTIONS COMPARISON


,Parameter,Sol 1,Sol 2,Sol 3,Sol 4,Sol 5,Sol 6,Sol 7,Sol 8,Sol 9,Sol 10,Sol 11,Sol 12
0,Composite Score,1.38,1.42,1.5,1.52,1.54,1.54,1.54,1.56,1.58,1.66,1.71,1.76
1,Mech Fail Prob,27.2%,50.8%,43.8%,43.1%,50.4%,58.3%,18.4%,69.9%,65.6%,94.9%,76.2%,43.7%
2,GAP Score,10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P),10.0 (SD) → 1 (P)
3,────────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────,──────────
4,SURGICAL PLAN,,,,,,,,,,,,
5,UIV_implant,Hook,Hook,PS,PS,FS,FS,Hook,PS,Hook,FS,FS,Hook
6,num_levels_cat,lower,higher,lower,lower,higher,higher,higher,lower,lower,lower,lower,higher
7,num_interbody_fusion_levels,4,4,4,5,5,4,4,5,5,5,5,4
8,ALIF,1,1,1,0,0,1,0,1,0,1,0,0
9,XLIF,0,1,0,1,1,0,0,1,0,1,0,1
